This file will be used to contain data processing components

In [1]:
# %pip install boto3 pyspark delta-spark python-dotenv py4j

In [2]:
#import necessary libraries
import os
from pyspark.sql import SparkSession

In [3]:
# You can use the following to set the environment variables in the notebook if you don't set manually access key, secret key and endpoint in minio
os.environ['OBJ_STORAGE_ACCESS_KEY'] = 'ZfDdneDqDMWO07RvroVX'
os.environ['OBJ_STORAGE_SECRET_KEY'] = 'v8V87ClJJwLAIqBPll1ix4V93Vgjdpbt0sowjt5k'
os.environ['OBJ_STORAGE_ENDPOINT'] = 'http://localhost:9000'

In [4]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT')

In [5]:
#path of data in minio
path_1 = "s3a://data/data-raw/data.json"
path_2 = "s3a://data/data-raw/data2.json"
path_3 = "s3a://data/data-raw/data3.json"

In [6]:
# Create a SparkSession with necessary configurations
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "ZfDdneDqDMWO07RvroVX") \
    .config("spark.hadoop.fs.s3a.secret.key", "v8V87ClJJwLAIqBPll1ix4V93Vgjdpbt0sowjt5k") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

In [7]:
# Read JSON files into Spark DataFrames
df_1 = spark.read.json(path_1)
df_2 = spark.read.json(path_2)
df_3 = spark.read.json(path_3)

In [8]:
# Merge the DataFrames and write the result to a JSON file
merged_result = df_1.union(df_2).union(df_3)
merged_result.write.format("json").mode('overwrite').save("s3a://data/data-result/result.json")

# Read the JSON file from S3 into a DataFrame
df_result = spark.read.json("s3a://data/data-result/result.json")
df_result.show(truncate=False)

# Drop duplicate rows from the DataFrame
df_result = df_result.dropDuplicates()
df_result.show(truncate=False)

# Write the deduplicated DataFrame to a Parquet file on S3
df_result.write.format("parquet").save("s3a://data/data-result/result.parquet")

+---------------+----+----+------------+
|_corrupt_record|id  |name|type        |
+---------------+----+----+------------+
|NULL           |NULL|NULL|NULL        |
|NULL           |NULL|NULL|NULL        |
|NULL           |NULL|NULL|NULL        |
|NULL           |NULL|NULL|NULL        |
|NULL           |NULL|NULL|NULL        |
|NULL           |NULL|NULL|NULL        |
|NULL           |NULL|NULL|NULL        |
|NULL           |NULL|NULL|NULL        |
|NULL           |1001|NULL|Regular     |
|NULL           |1002|NULL|Chocolate   |
|NULL           |1003|NULL|Blueberry   |
|NULL           |1004|NULL|Devil's Food|
|NULL           |NULL|NULL|NULL        |
|NULL           |NULL|NULL|NULL        |
|NULL           |NULL|NULL|NULL        |
|NULL           |5001|NULL|None        |
|NULL           |5002|NULL|Glazed      |
|NULL           |5003|NULL|Chocolate   |
|NULL           |5004|NULL|Maple       |
|NULL           |5005|NULL|Sugar       |
+---------------+----+----+------------+
only showing top